In [1]:
import pandas as pd
import os
import random

In [2]:
df = pd.read_csv('primaryschool.csv')

In [3]:
df.describe()

,Time,Person 1,Person 2
count,125773.000000,125773.000000,125773.000000
mean,90687.675574,1635.858109,1737.303897
std,43793.491033,122.818086,137.779067
min,31220.000000,1426.000000,1427.000000
25%,45640.000000,1545.000000,1653.000000
50%,118580.000000,1661.000000,1759.000000
75%,132020.000000,1719.000000,1857.000000
max,148120.000000,1920.000000,1922.000000


In [4]:
timestamps = (df["Time"].iloc[-1] - df["Time"].iloc[0])/20
len(df["Time"].unique())

3100

105 timestamps are going to be clustered together and considered as one day.<br>
This would make the dataset into 30 days

In [5]:
class SIR:
    suscepted = []
    infected = []
    recovered = []
    vaccinated = []
    # Can be modified according to the disease scenario
    alpha = 0.8
    beta = 0.3
    initial_infected = 12
    
#     As per the data in the dataset
    start_range = 1426
    end_range = 1922+1
    
    def random_sample(self,start_range,end_range,select):
        return random.sample(range(start_range,end_range),select)
    
    # Vaccinate the people in the list 'vaccinated_people'
    def vaccinate(self, vaccinated_people):
        for person in vaccinated_people:
            # Remove vaccinated person from suscepted people
            if person in self.suscepted:
                self.suscepted.remove(person)
                self.recovered.append(person)
            # Remove vaccinated person from infected people
            elif person in self.infected:
                self.infected.remove(person)
                self.recovered.append(person)

    def init(self):
        for person in range(self.start_range,self.end_range):
            self.suscepted.append(person)
        self.infected = self.random_sample(self.start_range,self.end_range,self.initial_infected)
        
        for infected_person in self.infected:
            self.suscepted.remove(infected_person)
    
    # determine which category person belongs to
    def person_type(self, person):
        if(person in self.suscepted):
            return 'suscepted'
        if(person in self.infected):
            return 'infected'
        return 'recovered'

    # Simulate new infected people
    def get_new_infected(self, infected_contact):
        total = len(infected_contact)
        new_infected_total = self.alpha*total
        new_infected = random.sample(infected_contact,k = int(new_infected_total))
        
        # Remove duplicates
        new_infected = list(set(new_infected))
        
        # Add them to infected
        for infected_person in new_infected:
            self.infected.append(infected_person)
            self.suscepted.remove(infected_person)
    
    # Simulate new recovered people
    def get_new_recovered(self):
        recovered_count = self.beta*(len(self.infected))
        new_recovered = random.sample(self.infected,k = int(recovered_count))
        
        # Add them to recovered
        for recovered_person in new_recovered:
            self.infected.remove(recovered_person)
            self.recovered.append(recovered_person)
        
        
        
        

In [6]:
# Vaccination strategy: Random 20% of people
def vaccination_strategy(model):
    vaccination_count = 0.2*(model.end_range - model.start_range)
    vaccinated = random.sample(range(model.start_range,model.end_range), int(vaccination_count))
    return vaccinated

In [7]:
model = SIR()
model.init()


In [8]:
vaccinated = vaccination_strategy(model)
model.vaccinate(vaccinated)

In [9]:
print("At day 0")
print("Number of susceptible: ",len(model.suscepted))
print("Number of infected: ",len(model.infected))
print("Number of recovered: ",len(model.recovered))

At day 0
Number of susceptible:  387
Number of infected:  11
Number of recovered:  99


In [10]:
total_count = 0
days = 0
previous_timestamp = 0


while total_count < df.shape[0]:
    count = 0
    infected_contact = []
    while count < 105 and total_count < df.shape[0]:
        person1 = df["Person 1"][total_count]
        person2 = df["Person 2"][total_count]
        
        # Check for transitions from Suscepted to Infected
        if model.person_type(person1) == 'suscepted' and model.person_type(person2) == 'infected':
            infected_contact.append(person1)
        if model.person_type(person2) == 'suscepted' and model.person_type(person1) == 'infected':
            infected_contact.append(person2)
        
        # If new timestamp, then increase counts
        if( df["Time"][total_count] != previous_timestamp):
            previous_timestamp = df["Time"][total_count]
            count = count + 1
        total_count = total_count + 1
    
    days = days + 1
    model.get_new_recovered()
    model.get_new_infected(infected_contact)
    print("After Day ",days)
    print("Number of susceptible: ",len(model.suscepted))
    print("Number of infected: ",len(model.infected))
    print("Number of recovered: ",len(model.recovered))
    
    
            
        
        

After Day  1
Number of susceptible:  366
Number of infected:  29
Number of recovered:  102
After Day  2
Number of susceptible:  330
Number of infected:  57
Number of recovered:  110
After Day  3
Number of susceptible:  276
Number of infected:  94
Number of recovered:  127
After Day  4
Number of susceptible:  209
Number of infected:  133
Number of recovered:  155
After Day  5
Number of susceptible:  202
Number of infected:  101
Number of recovered:  194
After Day  6
Number of susceptible:  202
Number of infected:  71
Number of recovered:  224
After Day  7
Number of susceptible:  202
Number of infected:  50
Number of recovered:  245
After Day  8
Number of susceptible:  201
Number of infected:  36
Number of recovered:  260
After Day  9
Number of susceptible:  201
Number of infected:  26
Number of recovered:  270
After Day  10
Number of susceptible:  201
Number of infected:  19
Number of recovered:  277
After Day  11
Number of susceptible:  201
Number of infected:  14
Number of recovered: 